In [1]:
# 1
# Chance Wiese

In [2]:
pip install pymssql


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: /Users/chancewiese/Local Documents/school/data-5330/pymssql_env/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# 2
import pandas as pd
import pymssql

conn = pymssql.connect(
     host='stairway.usu.edu', # Server name goes in quotes.
     user='dunn2100', # Username goes in quotes.
     password='databases4ever', # Password goes in quotes
     database='overland') # Database to use goes in quotes

query = """
With MemberCount AS (
    SELECT HHID, COUNT(*) AS Members
    FROM Guest
    GROUP BY HHID
),
SeasonPassCount AS (
    SELECT HHID, COUNT(*) AS SPCount
    FROM Guest
    WHERE SeasonPass = 'y'
    GROUP BY HHID
),
Under18Count AS (
    SELECT HHID, COUNT(*) AS U18Count
    FROM Guest
    WHERE Age < 18
    GROUP BY HHID
),
VisitCount AS (
    SELECT HHID, COUNT(DISTINCT VisID) AS VisitCt
    FROM Guest g
    JOIN Visit v ON g.GuestID = v.CustID
    GROUP BY g.HHID
)
SELECT h.HHID, Region, HHI AS Income, Lname AS Lname, m.Members, sp.SPCount, u.U18Count, v.VisitCt
FROM Household h
JOIN Guest g ON h.HHID = g.HHID
JOIN MemberCount m ON h.HHID = m.HHID
LEFT JOIN SeasonPassCount sp ON h.HHID = sp.HHID
LEFT JOIN Under18Count u ON h.HHID = u.HHID
LEFT JOIN VisitCount v ON h.HHID = v.HHID
GROUP BY h.HHID, Region, HHI, Lname, m.Members, sp.SPCount, u.U18Count, v.VisitCt
ORDER BY h.HHID;
"""

df = pd.read_sql(query, conn, index_col='HHID')
print(df)

OperationalError: (20009, b'DB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (stairway.usu.edu)\nNet-Lib error during Operation timed out (60)\nDB-Lib error message 20009, severity 9:\nUnable to connect: Adaptive Server is unavailable or does not exist (stairway.usu.edu)\nNet-Lib error during Operation timed out (60)\n')

In [ ]:
# 3
import numpy as np
df['SPCount'] = df['SPCount'].replace(np.nan, 0)
df['U18Count'] = df['U18Count'].replace(np.nan, 0)
df['VisitCt'] = df['VisitCt'].replace(np.nan, 0)
print(df)

In [ ]:
# 4
df['VisPP'] = df['VisitCt'] / df['Members']
print(df[['Lname', 'Members', 'VisitCt', 'VisPP']])

In [ ]:
# 5
df['IsLocal'] = np.where(
    df['Region'] == 'Central PA',
    1, 0)
print(df[['Lname', 'Region', 'IsLocal']])

In [ ]:
# 6
dflocal = df[df['IsLocal'] == 1]
print(dflocal)

In [ ]:
# 7
dflocal.drop(columns=['IsLocal', 'Region'], inplace=True)
print(dflocal)

In [ ]:
# 8
dflocal.dropna(inplace=True)

conditions = [
    dflocal['Income'] < 50000,
    dflocal['Income'] < 100000,
    dflocal['Income'] < 150000,
    True
]
values = ['Under $50K', '$50-100K', '$100-150K', 'Over $150K']
dflocal['IncomeCat'] = np.select(conditions, values)
print(dflocal[['Lname', 'Income', 'IncomeCat', 'VisPP']])

In [ ]:
# 9
analysis = dflocal[['VisPP', 'IncomeCat']].groupby('IncomeCat').agg(['mean', 'median', 'count'])
print(analysis, 
"""\n
Yes, there is a relationship between income and number of visits per person among local customers. 
The analysis shows that average VisPP consistently increases with income, from 3.89 for 'Under $50K' to 7.53 for 'Over $150K'.
This shows a positive correlation between income and visit frequency.
""")

In [ ]:
# 10
df_corr = dflocal[['Income', 'Members', 'SPCount', 'U18Count', 'VisPP']].corr()
print(df_corr['VisPP'],
"""\n
The variable with the strongest positive relationship to visits per person (VisPP) is Income (0.286).
The variable with the weakest relationship is Members (-0.118).
This shows that higher income is associated with more visits, while number of members has very little impact on visit frequency.
"""
)